In [3]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
import pandas as pn
from datetime import datetime
import gsw

In [2]:
#Shipdata
seawolf = pn.read_csv("~/Desktop/NYB Indicators/cold_pool/combined.CTD.datasheets-2018-2020.csv")

In [3]:
# parse multiple date formats

def try_parsing_date(text):
    for fmt in ('00%y-%m-%d', '%Y-%m-%d'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [4]:
year = np.empty(622)
month = np.empty(622)
for i in range(622):
    year[i] = try_parsing_date(str(seawolf['Date'][i])).year
    month[i] = try_parsing_date(str(seawolf['Date'][i])).month

In [5]:
seawolf['year'] = year
seawolf['month'] = month

In [6]:
seawolf = seawolf.rename(columns={'CTD.Depth..m.':'Depth','Temp...C.':'Temp'})
seawolf['Temp'][seawolf['Temp']=='?']=np.nan
seawolf['Temp'] = seawolf['Temp'].astype(float)
seawolf['Depth'] = seawolf['Depth'].astype(float)

<ipython-input-6-0ae08e00b5ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seawolf['Temp'][seawolf['Temp']=='?']=np.nan


In [10]:
seawolf = seawolf.rename(columns={'Lat.Start':'lat','Long.Start':'lon'})

In [11]:
seawolf

,Unnamed: 0,CruiseID,Date,Cast,Station,Bottle.No,lat,Lat.End,lon,Long.End,Depth,Temp,year,month
0,1,NYOS1807,0018-07-14,1,1.1,1,40.958537,40.57591,71.820132,71.49377,113.0,NaN,2018.0,7.0
1,2,NYOS1807,0018-07-14,1,1.1,2,40.958537,40.57591,71.820132,71.49377,50.0,NaN,2018.0,7.0
2,3,NYOS1807,0018-07-14,1,1.1,3,40.958537,40.57591,71.820132,71.49377,2.0,NaN,2018.0,7.0
3,4,NYOS1807,0018-07-14,2,1.2,1,40.417600,40.41412,71.430810,71.42659,62.3,NaN,2018.0,7.0
4,5,NYOS1807,0018-07-14,2,1.2,2,40.417600,40.41412,71.430810,71.42659,62.3,NaN,2018.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,618,NYOS2002,0020-02-10,13,6.2,62,40.101940,40.10348,73.195510,73.19446,10.0,6.435,2020.0,2.0
618,619,NYOS2002,0020-02-10,13,6.2,63,40.101940,40.10348,73.195510,73.19446,2.0,6.430,2020.0,2.0
619,620,NYOS2002,0020-02-10,14,6.1,64,40.307320,40.31049,73.266940,73.26432,15.0,5.510,2020.0,2.0
620,621,NYOS2002,0020-02-10,14,6.1,65,40.307320,40.31049,73.266940,73.26432,10.0,5.510,2020.0,2.0


In [23]:
#no salinity in this seawolf data - can do a mixed layer based on T (want to do pt but it needs salinity in the conversion...)
# loop through the years, find the unique casts in that year
for i in np.unique(seawolf.Cast):
    temp = seawolf.Temp[seawolf.Cast == i]
    depth = seawolf.Depth[seawolf.Cast==i]
    if any(temp) == 1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [37]:
seawolf[3:9].pressure.values


array([62.79288245, 62.79288245, 36.38327148, 24.38918391, 11.28719717,
        1.71319546])

In [21]:
from holteandtalley import HolteAndTalley

#h = HolteAndTalley(pressures,temperatures)
 ##The temperature algorithms mixed layer depth
#h.tempMLD

In [41]:
h = HolteAndTalley(seawolf[5:9].pressure.values,seawolf[5:9].Temp.values)

ValueError: zero-size array to reduction operation maximum which has no identity

In [42]:
seawolf[5:9].Temp.values

array([ 9.13,  9.35, 19.96, 21.77])

In [5]:
data = pn.DataFrame(data=np.empty([9,9]),columns = ['depth','temp','salt','lat','lon','year','month','day','profile#'])

In [6]:
import os

In [7]:
#Opening the CTD data 2018-2021 WOD
k=1
for i in os.listdir("/Users/nyelab/Downloads/ocldb1618533792.17769.CTD/"):
    if i.startswith("wo"):
        string = '/Users/nyelab/Downloads/ocldb1618533792.17769.CTD/'+i
        array = xr.open_dataset(string)
        data2 = pn.DataFrame(data = array.Temperature, columns = ['temp'])
        data2['salt'] = array.Salinity
        depth = array.z.values
        Lon = array.lon.values
        Lat = array.lat.values
        year = pn.to_datetime(array.time.values).year
        month = pn.to_datetime(array.time.values).month
        day = pn.to_datetime(array.time.values).day
        data2['depth'] = depth
        data2['year']=year
        data2['month']=month
        data2['day']=day
        data2['lat']=Lat
        data2['lon']=Lon
        data2['profile#']=k
        k=k+1
        data = data.append(data2)
        array.close()

In [87]:
test_profile = data[8:50]

In [93]:
data[data['profile#']==1]

,depth,temp,salt,lat,lon,year,month,day,profile#
0,0.992336,12.53,32.693001,40.0,-72.176666,2018.0,5.0,26.0,1.0
1,1.984667,12.53,32.693001,40.0,-72.176666,2018.0,5.0,26.0,1.0
2,2.976993,12.53,32.694000,40.0,-72.176666,2018.0,5.0,26.0,1.0
3,3.969314,12.52,32.695000,40.0,-72.176666,2018.0,5.0,26.0,1.0
4,4.961631,12.51,32.696999,40.0,-72.176666,2018.0,5.0,26.0,1.0
...,...,...,...,...,...,...,...,...,...
74,74.411827,7.64,33.620998,40.0,-72.176666,2018.0,5.0,26.0,1.0
75,75.403801,7.64,33.620998,40.0,-72.176666,2018.0,5.0,26.0,1.0
76,76.395782,7.64,33.619999,40.0,-72.176666,2018.0,5.0,26.0,1.0
77,77.387741,7.64,33.619999,40.0,-72.176666,2018.0,5.0,26.0,1.0


,depth,temp,salt,lat,lon,year,month,day,profile#
0,-1.727234e-77,-3.111085e+231,1.581010e-322,NaN,3.607058e-85,1.575887e-52,6.013470e-154,4.700750e+180,NaN
1,2.441852e-154,2.410794e+198,1.336537e-152,1.654872e-153,2.413176e+185,1.257682e-76,6.013345e-154,6.013470e-154,NaN
2,1.816679e-152,6.013470e-154,1.444204e+214,7.013726e-77,4.960421e+180,2.097754e-153,2.445159e-154,6.013470e-154,NaN
3,2.288961e+243,6.013345e-154,9.926786e+247,6.013470e-154,5.376250e+241,1.392378e-259,3.154755e+180,5.826596e+252,NaN
4,3.292655e+180,7.497753e+247,2.199803e-152,2.348396e+251,5.626309e+175,4.909001e+252,3.319231e+63,2.006847e+174,NaN
...,...,...,...,...,...,...,...,...,...
52,5.258790e+01,6.940000e+00,3.330700e+01,3.984500e+01,-7.283500e+01,2.018000e+03,5.000000e+00,2.600000e+01,493.0
53,5.358000e+01,6.940000e+00,3.330700e+01,3.984500e+01,-7.283500e+01,2.018000e+03,5.000000e+00,2.600000e+01,493.0
54,5.457208e+01,6.930000e+00,3.330700e+01,3.984500e+01,-7.283500e+01,2.018000e+03,5.000000e+00,2.600000e+01,493.0
55,5.556417e+01,6.940000e+00,3.330700e+01,3.984500e+01,-7.283500e+01,2.018000e+03,5.000000e+00,2.600000e+01,493.0


In [60]:
test_profile['pressure'] = gsw.p_from_z(-1*test_profile['depth'].values,test_profile['lat'].values)

<ipython-input-60-b635dab94144>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_profile['pressure'] = gsw.p_from_z(-1*test_profile['depth'].values,test_profile['lat'].values)


AttributeError: module 'gsw' has no attribute '_mlp'

In [65]:
test_profile['SA'] = gsw.SA_from_SP(test_profile.salt.values,test_profile.pressure.values, test_profile.lon.values, test_profile.lat.values)

<ipython-input-65-6eccd28d2a48>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_profile['SA'] = gsw.SA_from_SP(test_profile.salt.values,test_profile.pressure.values, test_profile.lon.values, test_profile.lat.values)


In [69]:
test_profile['CT'] = gsw.CT_from_t(test_profile.SA.values,test_profile.temp.values,test_profile.pressure.values)

<ipython-input-69-f60d317be9c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_profile['CT'] = gsw.CT_from_t(test_profile.SA.values,test_profile.temp.values,test_profile.pressure.values)


In [76]:
test_mld = HolteAndTalley(test_profile.pressure.values,test_profile.CT.values)

In [79]:
test_mld.tempMLD

10.497946033760718

In [80]:
test_profile

,depth,temp,salt,lat,lon,year,month,day,pressure,SA,CT
0,0.992336,12.53,32.693001,40.0,-72.176666,2018.0,5.0,26.0,0.999999,32.847918,12.569145
1,1.984667,12.53,32.693001,40.0,-72.176666,2018.0,5.0,26.0,1.999997,32.847918,12.569014
2,2.976993,12.53,32.694000,40.0,-72.176666,2018.0,5.0,26.0,2.999996,32.848921,12.568862
3,3.969314,12.52,32.695000,40.0,-72.176666,2018.0,5.0,26.0,3.999994,32.849925,12.558682
4,4.961631,12.51,32.696999,40.0,-72.176666,2018.0,5.0,26.0,4.999993,32.851932,12.548479
5,5.953942,12.50,32.702000,40.0,-72.176666,2018.0,5.0,26.0,5.999991,32.856956,12.538211
6,6.946249,12.41,32.712002,40.0,-72.176666,2018.0,5.0,26.0,6.999990,32.867005,12.447605
7,7.938551,12.21,32.737000,40.0,-72.176666,2018.0,5.0,26.0,7.999989,32.892120,12.246375
8,8.930849,11.95,32.766998,40.0,-72.176666,2018.0,5.0,26.0,8.999988,32.922261,11.984896
9,9.923141,11.58,32.803001,40.0,-72.176666,2018.0,5.0,26.0,9.999987,32.958434,11.613038


In [8]:
data['pressure'] = gsw.p_from_z(-1*data['depth'].values,data['lat'].values)

/Users/nyelab/miniconda3/lib/python3.8/site-packages/gsw/_wrapped_ufuncs.py:2312: RuntimeWarning: invalid value encountered in p_from_z
  return _gsw_ufuncs.p_from_z(z, lat, geo_strf_dyn_height, sea_surface_geopotential)


In [9]:
data['SA'] = gsw.SA_from_SP(data.salt.values,data.pressure.values, data.lon.values, data.lat.values)

In [14]:
data['CT'] = gsw.CT_from_t(data.SA.values, data.temp.values, data.pressure.values)

/Users/nyelab/miniconda3/lib/python3.8/site-packages/gsw/_wrapped_ufuncs.py:696: RuntimeWarning: invalid value encountered in ct_from_t
  return _gsw_ufuncs.ct_from_t(SA, t, p)


In [16]:
data['sigma'] = gsw.sigma0(data.SA.values, data.CT.values)

In [18]:
data

,depth,temp,salt,lat,lon,year,month,day,profile#,pressure,SA,CT,sigma
0,2.315842e+77,2.315842e+77,2.223295e-322,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,6.929307e-310,6.929307e-310,0.0,0.0,0.0,0.000000,0.000000,0.015284,-0.155536
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.015284,-0.155536
3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.015284,-0.155536
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.015284,-0.155536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,5.258790e+01,6.940000e+00,3.330700e+01,39.845001,-7.283500e+01,2.018000e+03,5.0,26.0,493.0,52.999929,33.465048,6.951143,26.096098
53,5.358000e+01,6.940000e+00,3.330700e+01,39.845001,-7.283500e+01,2.018000e+03,5.0,26.0,493.0,53.999929,33.465049,6.951053,26.096111
54,5.457208e+01,6.930000e+00,3.330700e+01,39.845001,-7.283500e+01,2.018000e+03,5.0,26.0,493.0,54.999918,33.465050,6.940952,26.097466
55,5.556417e+01,6.940000e+00,3.330700e+01,39.845001,-7.283500e+01,2.018000e+03,5.0,26.0,493.0,55.999928,33.465051,6.950872,26.096137


In [27]:
data['mldT']=''
data['mldS']=''
data['mldr']=''
for i in range(1,494):
    profile = data[data['profile#']==i]
    mlds = HolteAndTalley(profile.pressure.values,profile.temp.values,profile.salt.values,profile.sigma.values)
    data.mldT[data['profile#']==i] = mlds.tempMLD
    data.mldS[data['profile#']==i] = mlds.salinityMLD
    data.mldr[data['profile#']==i] = mlds.densityMLD

<ipython-input-27-a0717cfca7e0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.mldT[data['profile#']==i] = mlds.tempMLD
<ipython-input-27-a0717cfca7e0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.mldS[data['profile#']==i] = mlds.salinityMLD
<ipython-input-27-a0717cfca7e0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.mldr[data['profile#']==i] = mlds.densityMLD
<ipython-input-27-a0717cfca7e0>:7: SettingWithCopyW

ValueError: zero-size array to reduction operation maximum which has no identity

In [29]:
profile

,depth,temp,salt,lat,lon,year,month,day,profile#,pressure,SA,CT,sigma,mldT,mldS,mldr
0,0.992444,6.40,31.422001,38.798332,-74.626663,2019.0,3.0,29.0,5.0,0.999998,31.570969,6.439836,24.679733,,,
1,1.984882,6.40,31.417999,38.798332,-74.626663,2019.0,3.0,29.0,5.0,1.999997,31.566947,6.439805,24.676587,,,
2,2.977316,6.40,31.415001,38.798332,-74.626663,2019.0,3.0,29.0,5.0,2.999995,31.563934,6.439762,24.674233,,,
3,3.969746,6.41,31.413000,38.798332,-74.626663,2019.0,3.0,29.0,5.0,3.999994,31.561923,6.449746,24.671429,,,
4,4.962170,6.41,31.410999,38.798332,-74.626663,2019.0,3.0,29.0,5.0,4.999992,31.559911,6.449689,24.669861,,,
5,5.954589,6.40,31.417000,38.798332,-74.626663,2019.0,3.0,29.0,5.0,5.999991,31.565940,6.439489,24.675837,,,
6,6.947004,6.38,31.469999,38.798332,-74.626663,2019.0,3.0,29.0,5.0,6.999990,31.619189,6.418638,24.720104,,,
7,7.939414,6.44,31.565001,38.798332,-74.626663,2019.0,3.0,29.0,5.0,7.999988,31.714640,6.477551,24.787580,,,
8,8.931820,6.44,31.589001,38.798332,-74.626663,2019.0,3.0,29.0,5.0,8.999987,31.738753,6.477154,24.806510,,,
9,9.924220,6.31,31.655001,38.798332,-74.626663,2019.0,3.0,29.0,5.0,9.999986,31.805065,6.345727,24.874652,,,
